# Code to normalize the images
April 7, 2020



In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

import subprocess as sp
import os
import sys
import time

In [2]:
%matplotlib widget

In [3]:
sys.path.append('/global/u1/v/vpa/project/jpt_notebooks/Cosmology/Cosmo_GAN/LBANN/lbann_cosmogan/3_analysis/')
from modules_image_analysis import *

[NbConvertApp] Converting notebook modules_image_analysis.ipynb to script
[NbConvertApp] Writing 8876 bytes to modules_image_analysis.py


In [ ]:
data_dir='/global/project/projectdirs/dasrepo/vpa/supernova_cnn/data/gathered_data/'

In [ ]:
fname1=data_dir+'summary_label_files.csv'
df1=pd.read_csv(fname1,sep=',',comment='#')
print(df1.shape)

In [ ]:
num_sig,num_bkgnd=df1[df1.Label==1].shape[0],df1[df1.Label==0].shape[0]
print("Proportion of Signal-Background: {0}-{1}.\nProportion of Signal: {2}".format(num_sig,num_bkgnd,num_sig*1.0/(num_sig+num_bkgnd)))

### Extract a slice of data

In [ ]:
### Extracting 30 images of signal and bkgnd
size= 3000
df_sig=df1[df1.Label==1].head(size)
df_bkg=df1[df1.Label==0].head(size)

del(df1)

In [ ]:
#df_sig
# df_bkg

## Extract image arrays

In [ ]:
def f_get_image_arr(df,mode='type',ftype='diff',idx=5):
    '''
    Module to get image arrays from dataframe with filenames
    Input: Dataframe, mode
    2 modes: 
    'type': Gives all the images for the same type of files,
    'index': Gives all 3 types for the same index
    'ftype': type of files to extract : srch, temp, diff
    'idx': index number of ID array from which to extract
    '''
    
    if mode=='type': ### Pick all images of type=ftype
        df2=df[df.filename.str.contains(ftype)].reset_index(drop=True)
        ### Read .gif files and store them in an array
        imgs=[plt.imread(fle) for fle in df2['file path']]
        
    elif mode=='index': ### Pick srch','temp','diff'
        index=np.unique(df_sig.ID.values)[idx]
        df2=df[df.ID==index].reset_index(drop=True)
        imgs=[plt.imread(fle) for fle in df2['file path']]
    
    df2.loc[:,'image']=imgs
    return df2


# df=f_get_image_arr(df_sig,mode='index',idx=0)
# df=f_get_image_arr(df_sig,mode='type',ftype='diff')


In [ ]:
df=f_get_image_arr(df_sig,mode='type',ftype='temp')
img_arr=np.stack(df.image.values)
f_pixel_intensity(img_arr,normalize=False)

In [ ]:

def f_compare_pixel_intensity_images(df_input,title,mode='normal'):
    '''
    Compare pixel intensity histogram of all 3 files.
    2 modes: 
        normal: takes all values and computes histogram
        averaged: takes histograms for each images and computes mean and error
    '''
    
    plt.figure()

    for ftype in['srch','temp','diff']:
        df=f_get_image_arr(df_input,mode='type',ftype=ftype)
        img_arr=np.stack(df.image.values)  ### Extract the image array samples
        
        norm=True
        if mode=='normal':
            hist, bin_edges = np.histogram(img_arr.flatten(), bins=25, density=norm)
            centers = (bin_edges[:-1] + bin_edges[1:]) / 2
            #     print(bin_edges,centers)
            plt.errorbar(centers, hist, fmt='o-', label=ftype)

        elif mode=='avg':
            hist_arr=np.array([np.histogram(arr.flatten(), bins=25, density=norm) for arr in img_arr])
            hist=np.stack(hist_arr[:,0])
            bins=np.stack(hist_arr[:,1])
            ### Compute statistics of histogram of each image
            mean,err=np.mean(hist,axis=0),np.std(hist,axis=0)/np.sqrt(hist.shape[0])
            bin_edges=bins[0]
            centers = (bin_edges[:-1] + bin_edges[1:]) / 2

            plt.errorbar(centers,mean,yerr=err,fmt='o-',label=ftype)
        
        
    plt.xlabel('Pixel value')
    plt.ylabel('Counts')
    plt.title('Pixel Intensity Histogram of '+title)
    plt.legend(loc='best')
    
    
f_compare_pixel_intensity_images(df_sig,'signal',mode='avg')

In [ ]:
f_compare_pixel_intensity_images(df_bkg,title='bkgnd',mode='avg')

### Plot power spectrum

In [ ]:
# f_compute_spectrum(img_arr)

In [ ]:
# df=f_get_image_arr(df_sig,mode='index',idx=0)
# df

In [ ]:
imgs=df.image.values

### Implementing MAD: Median Absolute Deviation
https://en.wikipedia.org/wiki/Median_absolute_deviation

$ MAD=Median \left(X-\tilde{X} \right) \ \ $   where $ \tilde{X}$ is the median of array

$ \sigma = k . MAD $

In [ ]:
def f_mad(arr):
    '''
    Compute MAD and std
    '''
    arr2=arr.flatten()
    MD=np.median(arr2)
#     print(MD)
    mad=np.median(np.abs(arr2-MD))
    k=1.4826 ### For normal distribution
    sigma=mad*k

    return mad,sigma

In [ ]:
arr=img_arr[:10]
f_mad(arr)

In [ ]:
arr2=arr.flatten()
MD=np.median(arr2)
print(MD)
arr3=np.abs(arr2-MD)

## Implement normalization with actual data

In [10]:
arr=np.load('/global/project/projectdirs/dasrepo/vpa/supernova_cnn/data/gathered_data/input_npy_files/full_x.npy')
samples=arr[:]
# del arr

In [7]:
def f_mad(arr):
    '''
    Compute MAD and std
    '''
#     print(arr.shape)
    arr2=arr.flatten()
    MD=np.median(arr2)
#     print(MD)
    mad=np.median(np.abs(arr2-MD))
    k=1.4826 ### For normal distribution
    sigma=mad*k

    return mad,sigma

def f_vectorize(arr):
    return np.vectorize(f_mad)(arr)



# print(samples.shape)
# t1=time.time()
# scaled_samples=np.array([(1.0/f_mad(i[:,:,2])[1])*i for i in samples])
scales_arr=np.array([(1.0/f_mad(i[:,:,2])[1]) for i in samples])

# t2=time.time()

In [78]:
def f_scale(arr):
    '''
    
    '''
    samples=arr[:]
    s2=samples[:,:,:,2].reshape(N,51*51)
    scales=np.apply_along_axis(f_mad,1,s2)
#     print(scales.shape,samples.shape)
    scaled_arr=np.einsum('i,ijkl->ijkl',(1.0/scales),samples)
    
    return scaled_arr

def f_rescale_samples(samples):
    ''' Rescale individual images with MAD value of diff image
    '''
    def f_mad(arr):
        '''
        Compute MAD and std
        '''
        arr2=arr.flatten()
        MD=np.median(arr2)
    #     print(MD)
        mad=np.median(np.abs(arr2-MD))
        k=1.4826 ### For normal distribution
        sigma=mad*k

        return mad,sigma

    scaled_samples=np.array([(1.0/f_mad(i[:,:,2])[1]+1e-6)*i for i in samples])

#     scales=np.apply
#     scaled_2=np.einsum('i,ijkl->ijkl',,samples)
    return scaled_samples


# f_scale(arr[:20])

Procedure for normalization : 

For each sample (3 image types)
- Computed sigma using MAD method on diff image
- Divide entire sample by that value


In [22]:
save_location='/global/project/projectdirs/dasrepo/vpa/supernova_cnn/data/gathered_data/input_npy_files/'
f1='full_x.npy'
f2='renorm_full_x.npy'

ip_arr=np.load(save_location+f1)
print(ip_arr.shape)

def f_rescale_samples(samples):
    ''' Rescale individual images with MAD value of diff image
    '''
    def f_mad(arr):
        '''
        Compute MAD and std
        '''
        arr2=arr.flatten()
        MD=np.median(arr2)
    #     print(MD)
        mad=np.median(np.abs(arr2-MD))
        k=1.4826 ### For normal distribution
        sigma=mad*k

        return mad,sigma
    
    
    scaled_samples=np.ones_like(samples)
    lst_zeros=[] # List to store indices where the MAD value is zero
    for i,row in enumerate(samples):
        scale=f_mad(row[:,:,2])[1]
        if scale<1e-10: 
            print("Small value",i,scale)
            print(i,row.shape,f_mad(row[:,:,0]),f_mad(row[:,:,1]),f_mad(row[:,:,2]))
            lst_zeros.append(i)
            scale=1.0
        scaled_samples[i]=row*(1.0/scale)
        
        
    ### For every row, compute the MAD value for diff image (idx =2 ) and multiple its inverse to each sample
#     scaled_samples=np.array([(1.0/f_mad(i[:,:,2])[1]+1e-6)*i for i in samples])
    
    return scaled_samples,lst_zeros

t1=time.time()
rescaled_arr,zero_lst=f_rescale_samples(ip_arr[:10000])
t2=time.time()
print(t2-t1)

print(rescaled_arr.shape)
# np.save(save_location+f2,rescaled_arr)

(898963, 51, 51, 3)
Small value 1808 0.0
1808 (51, 51, 3) (35.0, 51.891) (36.0, 53.373599999999996) (0.0, 0.0)
Small value 1856 0.0
1856 (51, 51, 3) (5.0, 7.412999999999999) (13.0, 19.273799999999998) (0.0, 0.0)
Small value 3176 0.0
3176 (51, 51, 3) (1.0, 1.4826) (2.0, 2.9652) (0.0, 0.0)
Small value 5066 0.0
5066 (51, 51, 3) (50.0, 74.13) (54.0, 80.0604) (0.0, 0.0)
1.8579940795898438
(10000, 51, 51, 3)


In [21]:
for idx in zero_lst:
    img=ip_arr[idx][:,:,2]
    plt.figure()
    plt.imshow(img)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [93]:
# a2=np.load(save_location+f2)
# print(a2.shape)

(10000, 51, 51, 3)


In [5]:
def f_mad(arr):
    '''
    Compute MAD and std
    '''
    arr2=arr.flatten()
    MD=np.median(arr2)
#     print(MD)
    mad=np.median(np.abs(arr2-MD))
    k=1.4826 ### For normal distribution
    sigma=mad*k

    return mad,sigma


In [19]:
f_mad(ip_arr[1808,:,:,2])

(0.0, 0.0)

In [11]:
zero_lst

[1808, 1856, 3176, 5066]